# Deep Learning Analytics challenge report

In this report I discuss how I addressed the DLA challenge. The report starts with the my configuration/devops of the TK1. Next is a section on a simple problem of categorizing handwritten digits based on code and an example from Nielsen's ["Neural Networks and Deep Learning"](http://neuralnetworksanddeeplearning.com/). Finally I report on a partial solution to a more difficult problem of applying a neural network to a physical simulation. Specifically, approximating the value of the so-called maximum motive of a thermoelectron engine.

TK1 configuraiton and devops
============================
The TK1 ships with an installation of ubuntu; there's a user named ubuntu with password ubuntu.
I plugged the TK1 into the power supply and connected it via an ethernet cable to the network; all communication with the TK1 was performed via SSH.
According to the setup instructions, I must [execute a command so that `apt` doesn't clobber `libglx.so`](http://elinux.org/Jetson_TK1#An_important_step_before_connecting_the_Jetson_to_Internet) **before** connecting the board to the internet. Therefore, I unplugged my router from my cable modem.

For the sake of convenience and good systems administration, I wanted to configure the TK1 so that I could log in without a password using an SSH key.
I also wanted a user named "jrsmith3" to match the username on my notebook in which I will install all of the code and administer the machine.
Finally, I wanted to change the default password for the "ubuntu" user for the sake of security.

Change `ubuntu` user password
-----------------------------
First, I logged into the TK1 via the user "ubuntu."
I used the [passwordstore utility](http://www.passwordstore.org/) on my notebook to generate a difficult-to-guess password for this machine.

password: U33HzPE-(.vIQJ\:3kj_


Prevent `apt` from clobbering `libglx.so`
-----------------------------------------
I just [followed the instructions](http://elinux.org/Jetson_TK1#An_important_step_before_connecting_the_Jetson_to_Internet). I also plugged the router back into the cable modem and I'm back on the internet.


Create a user `jrsmith3`
------------------------
Pretty simple.

```
$ sudo adduser jrsmith3
Adding user `jrsmith3' ...
Adding new group `jrsmith3' (1001) ...
Adding new user `jrsmith3' (1001) with group `jrsmith3' ...
Creating home directory `/home/jrsmith3' ...
Copying files from `/etc/skel' ...
Enter new UNIX password: 
Retype new UNIX password: 
passwd: password updated successfully
Changing the user information for jrsmith3
Enter the new value, or press ENTER for the default
    Full Name []: Joshua Ryan Smith
    Room Number []: 
    Work Phone []: 
    Home Phone []: 
    Other []: joshua.r.smith@gmail.com
Is the information correct? [Y/n] y
Adding new user `jrsmith3' to extra groups ...
Adding user `jrsmith3' to group `video' ...
```


Give user `jrsmith3` sudo access
--------------------------------
For sysadmin purposes.

```
ubuntu@tegra-ubuntu:~$ sudo usermod -a -G sudo jrsmith3
```


Allow ssh logins via ssh keys
-----------------------------
Maybe this feature is already set up. I will try to copy my public ssh key from gamma over to the TX1. First, I had to create an `.ssh` directory on the TX1.

```
jrsmith3@tegra-ubuntu:~$ mkdir .ssh
```

Then I use scp to copy my ssh key to the TX1.

```
gamma:$ scp -rCp ~/.ssh/id_rsa.pub jrsmith3@tegra-ubuntu:.ssh
jrsmith3@tegra-ubuntu's password: 
id_rsa.pub                                    100%  396     0.4KB/s   00:00
```

After copying the public key, I added it to the `~/.ssh/authorized_keys` file.


Update apt and packages
-----------------------
Simple.

```
sudo apt-add-repository universe
sudo apt-get update
```

I will also install `bash-completion` and `command-not-found` as suggested in the instructions.

At this point I'm going to have to modify files in `/etc` and so I should probably set up `etckeeper`. To do that, I should also probably update the packages.


Install `etckeeper`, `git`, and other utils
-------------------------------------------
I need to modify some things in `~etc`, so out of an abundance of caution I will [install `etckeeper`](https://help.ubuntu.com/lts/serverguide/etckeeper.html) as well.

```
sudo apt-get install etckeeper
```

I'm pretty sure the above pulled in `bzr` and set up `/etc` as a `bzr` repo. [I want git instead](http://evilrouters.net/2011/02/18/using-etckeeper-with-git-on-ubuntu/).

```
sudo apt-get install -y git-core
```

Configure git

```
$ git config --global user.name "Joshua Ryan Smith"
$ git config --global user.email joshua.r.smith@gmail.com
```

After the above commands, I modified `/etc/etckeeper/etckeeper.conf` to use git. I had to unintialize `etckeeper` so that it would remove the bzr repo and use git instead.

```
$ sudo etckeeper uninit
[sudo] password for jrsmith3: 
** Warning: This will DESTROY all recorded history for /etc,
** including the bzr repository.

Are you sure you want to do this? [yN] y
Proceeding..
```

Next, I initialize `etckeeper`.

```
$ sudo etckeeper init
Initialized empty Git repository in /etc/.git/
$ sudo etckeeper commit "Initial commit."
```

Now everything looks right.


Upgrade packages
----------------
Simple.

```
sudo apt-get upgrade
```

# Install Anaconda Python distribution for development

The [anaconda python distribution](https://www.continuum.io/downloads) provided by Continuum.io makes managing python packages and environments very simple -- particularly when it comes to installing the typical numerical stack such as `numpy`, `scipy`, etc.

Continuum has an [anaconda for armv7l](https://www.continuum.io/content/conda-support-raspberry-pi-2-and-power8-le) which is the architecture of the TK1:

```
$ uname -m
armv7l
```

I will need to install miniconda, but `numpy`, `scipy`, and a few other packages have been built. I'm using the [latest armv7l](http://repo.continuum.io/miniconda/Miniconda-latest-Linux-armv7l.sh). 

The md5sum matches, next to install:

```
$ bash Miniconda-latest-Linux-armv7l.sh

# ...
# Superfluous output
# ...

installation finished.
Do you wish the installer to prepend the Miniconda install location
to PATH in your /home/jrsmith3/.bashrc ? [yes|no]
[no] >>> yes

Prepending PATH=/home/jrsmith3/miniconda/bin to PATH in /home/jrsmith3/.bashrc
A backup will be made to: /home/jrsmith3/.bashrc-miniconda.bak


For this change to become active, you have to open a new terminal.

Thank you for installing Miniconda!
```

The installer even backed up my old `.bashrc` file. I logged out and rebooted because ubuntu said I should.

# Creating python environments for development

Instead of installing packages system-wide, it is good development practice to create a virtual environment and install the necessary packages there. Additionally, some of my packages have been built using conda and can be installed into such an environment.

Creating an environment on the TK1 with conda is straightforward. First, execute the command to create a barebones python environment in the `env` subdirectory of the current directory:

```bash
conda create -yp ./env python
```

Next, install the `ipython` shell and my `tec` package into the environment.

```bash
conda install -y -c jrsmith3 tec ipython -p ./env
```

The `-c jrsmith3` flag tells conda to search my channel on anaconda.org for the `tec` package.

Finally, the environment can be activated

```bash
source activate ./env
```

# Easy problem: Neural Networks and Deep Learning

Michael Nielsen has written a very nice [book](http://neuralnetworksanddeeplearning.com/) introducing Neural Networks and Deep Learning. This book comes with a [repository](https://github.com/mnielsen/neural-networks-and-deep-learning) which includes code implementing various neural networks as well as example data for training and validation. Specifically, the repo contains the [MNIST data set](http://yann.lecun.com/exdb/mnist/) which features more than 10,000 digitized, handwritten digits and their correct classification. Implementing a neural network to classify these digits is as simple as following the code on the book's website.

First, clone the repo and change directories into it (on the TK1):

```bash
$ pwd
/home/jrsmith3/Documents

$ git clone https://github.com/mnielsen/neural-networks-and-deep-learning.git
$ cd neural-networks-and-deep-learning
```

Next, create a conda environment in the root directory of the repo and install numpy, scipy, and ipython.

```bash
$ conda create -yp ./env python
$ conda install -y numpy scipy ipython -p ./env
```

The following code is located at `/home/jrsmith3/Documents/neural-networks-and-deep-learning/src/easy_problem.py` and can be run once the conda environment has been activated. Note that neither `easy_problem.py` nor the conda environment in `env` has been checked into the `neural-networks-and-deep-learning` repository on the TK1.

In [3]:
import mnist_loader
import network
import time

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = network.Network([784, 30, 10])

start_time = time.time()
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)
print time.time() - start_time

Epoch 0: 8979 / 10000
Epoch 1: 9140 / 10000
Epoch 2: 9232 / 10000
Epoch 3: 9313 / 10000
Epoch 4: 9313 / 10000
Epoch 5: 9365 / 10000
Epoch 6: 9363 / 10000
Epoch 7: 9322 / 10000
Epoch 8: 9388 / 10000
Epoch 9: 9410 / 10000
Epoch 10: 9416 / 10000
Epoch 11: 9417 / 10000
Epoch 12: 9402 / 10000
Epoch 13: 9394 / 10000
Epoch 14: 9428 / 10000
Epoch 15: 9435 / 10000
Epoch 16: 9446 / 10000
Epoch 17: 9443 / 10000
Epoch 18: 9419 / 10000
Epoch 19: 9468 / 10000
Epoch 20: 9443 / 10000
Epoch 21: 9466 / 10000
Epoch 22: 9467 / 10000
Epoch 23: 9453 / 10000
Epoch 24: 9467 / 10000
Epoch 25: 9472 / 10000
Epoch 26: 9456 / 10000
Epoch 27: 9439 / 10000
Epoch 28: 9439 / 10000
Epoch 29: 9424 / 10000
316.686872005


As shown above, my MacBook Air required about 5 minutes for training and correctly identified 95% of the digits in the best case scenario. The TK1 obtained similar success in identifying the digits but required around 12 minutes to complete the training runs. Typical output for the training run on the TK1 is as follows:

```
In [1]: %run easy_problem.py
Epoch 0: 9032 / 10000
Epoch 1: 9229 / 10000
Epoch 2: 9239 / 10000
Epoch 3: 9377 / 10000
Epoch 4: 9397 / 10000
Epoch 5: 9388 / 10000
Epoch 6: 9450 / 10000
Epoch 7: 9449 / 10000
Epoch 8: 9459 / 10000
Epoch 9: 9483 / 10000
Epoch 10: 9470 / 10000
Epoch 11: 9508 / 10000
Epoch 12: 9493 / 10000
Epoch 13: 9535 / 10000
Epoch 14: 9500 / 10000
Epoch 15: 9504 / 10000
Epoch 16: 9505 / 10000
Epoch 17: 9492 / 10000
Epoch 18: 9538 / 10000
Epoch 19: 9519 / 10000
Epoch 20: 9520 / 10000
Epoch 21: 9531 / 10000
Epoch 22: 9525 / 10000
Epoch 23: 9514 / 10000
Epoch 24: 9523 / 10000
Epoch 25: 9518 / 10000
Epoch 26: 9516 / 10000
Epoch 27: 9543 / 10000
Epoch 28: 9525 / 10000
Epoch 29: 9524 / 10000
678.137501001
```

# More difficult problem: Predicting behavior of a complicated physical system

Neural networks are typically employed in computer vision applications such as image classification (i.e. finding pictures of dogs or cats) and handwriting recognition like the example above. 
Although NNs seem to mostly be used for image processing applications, there are some reports of applying NNs to problems in physics.
For example, [Grzeszczuk et.al.](http://dx.doi.org/10.1145/280814.280816) used a neural network to generate physically plausible computer animations.
After training, their network was able to produce physically realistic motion faster than the approach of solving the system's equations of motion.

If a neural network can be trained to approximate solutions to problems of motion faster than a full simulation, then it stands to reason that a neural network could be trained to control a robot or autonomous vehicle (e.g. drone aircraft, driverless car, etc.).
Consider the case where a neural network controls a driverless car; one likely task would be to design control system to avoid pedestrians.
Building a prototype driverless car and training course would be costly in terms of money as well as time: each training run would occur in realtime.
On the other hand, despite the computational costs, such training runs could be simulated by solving the relevant physical equations in order to avoid the costly (in terms of money) prototype buildout.
Indeed, [Ford employs exactly this strategy for its autonomous vehicle program](http://spectrum.ieee.org/view-from-the-valley/transportation/self-driving/autonomous-vehicles-learn-by-playing-video-games).

Given the limited amount of time to work on this project, I chose a much simpler problem to solve which is based on a physical simulation.
Specifically the task is to determine the value of the maximum motive for a thermoelectron engine given the operating parameters of the device.
This problem is good because it is not trivial but it is not as complicated as the autonomous vehicle examples described above.
Moreover, I've written code to perform this simulation and so it is suited for the problem of generating training data for the neural network.

## Background on thermoelectron engine

A thermoelectron engine, also known as a thermoelectron energy converter or TEC, is a thermodynamic heat engine consisting of two electrodes enclosed in an evacuated container.
The electrodes are separated by a distance named the interelectrode spacing.
The emitter electrode is held at a higher temperature than the collector electrode, and electrons are emitted via the phenomenon of thermoelectron emission.
These thermoelectrons travel across the interelectrode space and arrive at the collector where they are absorbed. 
The electrons then travel through a lead, through an external load where work is done, and back to the emitter to complete the circuit.

![Schematic-tec](tec_schematic.png)

The interesting output parameters of this device, such as the output power density and efficiency, are fully determined by the electron transport across the device.
Specifically, the output parameters are determined by a quantity known as the maximum motive of the device.
This quantity is essentially the highest barrier electrons encounter as they traverse the device.
There is a difficulty in calculating the maximum motive because the electron transport cannot be expressed by a closed-form equation.
Instead, a set of coupled differential equations must be self-consistently solved.

The [`tec`](https://github.com/jrsmith3/tec) python module I wrote implements a numerical solution to the electron transport problem described above and is intended to be used as the simulator to generate the training and validation data for the neural network.

## Installing `tec` on the TK1

ADD COPY

In [85]:
import itertools
import time
import json
import sys
import tec
import numpy as np
import network

In [5]:
em_params = {"temp": 1000.,
             "barrier": 2.,
             "richardson": 10., }

co_params = {"temp": 300.,
             "barrier": 1.,
             "richardson": 10.,
             "position": 10., }

emitter_temps = [("emitter", "temp", val) for val in np.linspace(300, 2000, 10)]
emitter_barriers = [("emitter", "barrier", val) for val in np.linspace(1, 4, 10)]
collector_barriers = [("collector", "barrier", val) for val in np.linspace(1, 4, 10)]
collector_voltages = [("collector", "voltage", val) for val in np.linspace(0, 2, 20)]
collector_positions = [("collector", "position", val) for val in np.linspace(1, 10, 5)]

In [6]:
# There's a better way to do this with iterators
data = []

start_time = time.time()
for itr in itertools.product(emitter_temps, emitter_barriers, collector_barriers, collector_voltages, collector_positions):
    # Construct dictionaries out of the combination of parameters
    em_dict = dict([itm[1:] for itm in itr if itm[0] == "emitter"])
    co_dict = dict([itm[1:] for itm in itr if itm[0] == "collector"])
    
    # Update the dictionaries used to initialize the TEC electrodes
    em_params = dict(em_params.items() + em_dict.items())
    co_params = dict(co_params.items() + co_dict.items())
    
    # Create electrodes and `Langmuir` objects
    em = tec.electrode.Metal.from_dict(em_params)
    co = tec.electrode.Metal.from_dict(co_params)
    
    data.append(tec.models.Langmuir(em, co))
    
print "Initialization time for list of Langmuir objects:"
print time.time() - start_time

10000
20000
30000
40000
50000
60000
70000
80000
90000
Initialization time for list of Langmuir objects:
4860.82978487


In [7]:
start_time = time.time()
with open("data.json", "w") as f:
    f.write(json.dumps(data, default=tec.io.to_json, indent=4))

print "Write time of JSON formatted file:"
print time.time() - start_time

Write time of JSON formatted file:
4984.23444796


In [8]:
print "Length of data array:"
print len(data)
print "Size of data array in memory"
print sys.getsizeof(data)

Length of data array:
100000
Size of data array in memory
824472


# Set up and train Neural Network
The following code converts the data calculated above to a form that can be used by [Nielsen's neural network code](https://github.com/mnielsen/neural-networks-and-deep-learning). Additionally, the network is set up and trained.

In [42]:
with open("data.json", "r") as f:
    complete_data = json.load(f)

In [72]:
def convert_dat(datum):
    """
    Convert dict data output by tec to tuple for neural net
    
    This function converts dictionary data output by the `tec` module
    to the 2-tuple format required by the NNADL [1] code.
    The zeroth element of the 2-tuple is a numpy array containing the
    independent variables, and the first element is the dependent variable.
    This function orders the independent variables from the dict to the 
    numpy array as follows:
    
    * collector barrier
    * collector temp
    * collector voltage
    * collector position
    * collector richardson
    * collector emissivity
    * emitter barrier
    * emitter temp
    * emitter voltage
    * emitter position
    * emitter richardson
    * emitter emissivity
    
    [1] https://github.com/mnielsen/neural-networks-and-deep-learning
    """
    electrodes = ("collector", "emitter")
    attributes = ("barrier", "temp", "voltage", "position", "richardson", "emissivity")
    
    abscissae = np.array([datum[electrode][attribute] for electrode, attribute in itertools.product(electrodes, attributes)])
    abscissae_matrix = np.ndarray(buffer=abscissae, shape=(12, 1))
    ordinate = datum["max_motive"]
    
    return (abscissae_matrix, ordinate)

In [73]:
training_data = [convert_dat(datum) for datum in complete_data[0:50000]]
test_data = [convert_dat(datum) for datum in complete_data[50001:80000]]

In [76]:
net = network.Network([12, 30, 1])

In [78]:
net.SGD(training_data, 10, 10, 3.0)

Epoch 0 complete
Epoch 1 complete
Epoch 2 complete
Epoch 3 complete
Epoch 4 complete
Epoch 5 complete
Epoch 6 complete
Epoch 7 complete
Epoch 8 complete
Epoch 9 complete
